<a href="https://colab.research.google.com/github/tamara-kostova/MSc_Thesis_Neuroimaging/blob/master/02_data_preprocessing_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing and Standardization

This notebook applies a unified preprocessing pipeline to all datasets
identified during exploratory analysis. The goal is to produce a clean,
consistent dataset suitable for CNNs, SAM-based segmentation, and
vision-language models.

In [1]:
import os
import glob
import shutil
import hashlib
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocessing Decisions
- All images converted to **grayscale**
- Resized to **224 × 224**
- Stored as **uint8 [0–255]**
- Normalization to [0,1] applied **at training time**
- Filenames made collision-safe via hashing

In [9]:
BASE_DIR = "/content/drive/MyDrive/MSc_Thesis_Neuroimaging"
RAW_DIR = f"{BASE_DIR}/data/raw"
PROC_DIR = f"{BASE_DIR}/data/processed"
SPLIT_DIR = f"{BASE_DIR}/data/split"

In [5]:
def safe_filename(filepath):
    h = hashlib.md5(filepath.encode()).hexdigest()[:8]
    return f"{h}_{os.path.basename(filepath)}"


## GRAYSCALE + STANDARDIZATION PIPELINE

In [6]:
def preprocess_image(filepath, target_size=(224, 224)):
    try:
        img = Image.open(filepath)

        # Convert all formats to grayscale
        if img.mode != "L":
            img = img.convert("L")

        img = img.resize(target_size, Image.Resampling.LANCZOS)

        return np.array(img, dtype=np.uint8)

    except Exception as e:
        print(f"⚠️ Failed: {filepath} | {e}")
        return None


In [7]:
def list_images(folder):
    files = []
    for ext in ("*.png", "*.jpg", "*.jpeg"):
        files.extend(glob.glob(os.path.join(folder, ext)))
    return files

## Batch preprocessing

In [8]:
def batch_preprocess_dataset(src_base, dst_base):
    """
    Preserve class structure while preprocessing
    """
    print(f"\n🔄 Preprocessing: {src_base}")
    os.makedirs(dst_base, exist_ok=True)

    total = 0

    for class_name in os.listdir(src_base):
        class_src = os.path.join(src_base, class_name)
        if not os.path.isdir(class_src):
            continue

        class_dst = os.path.join(dst_base, class_name)
        os.makedirs(class_dst, exist_ok=True)

        files = glob.glob(f"{class_src}/*.[jp][pn]g")
        print(f"  {class_name}: {len(files)} files")

        for f in tqdm(files, leave=False):
            img = preprocess_image(f)
            if img is not None:
                Image.fromarray(img).save(
                    os.path.join(class_dst, os.path.basename(f))
                )
                total += 1

    print(f"✓ Total processed: {total}")
    return total


## Run preprocessing

In [10]:
datasets = [
    ("MRI/tumor_binary", "MRI/tumor_binary_norm"),
    ("MRI/tumor_multiclass", "MRI/tumor_multiclass_norm"),
    ("MRI/MS", "MRI/ms_norm"),
    ("CT/stroke_binary", "CT/stroke_binary_norm"),
]

for src, dst in datasets:
    batch_preprocess_dataset(
        f"{PROC_DIR}/{src}",
        f"{PROC_DIR}/{dst}"
    )



🔄 Preprocessing: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/processed/MRI/tumor_binary
  tumor: 1500 files


  normal: 1500 files


✓ Total processed: 3000

🔄 Preprocessing: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/processed/MRI/tumor_multiclass
  Carcinoma: 23 files


  Schwannoma: 200 files


  Germinoma: 33 files


...

✓ Total processed: 2333

🔄 Preprocessing: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/processed/MRI/MS
  MS: 2657 files


  Control: 1246 files


✓ Total processed: 3903

🔄 Preprocessing: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/processed/CT/stroke_binary
  stroke: 2223 files


  normal: 4427 files


✓ Total processed: 6650


### Run preprocessing for jpegs

In [12]:
def batch_preprocess_dataset_jpegs(src_base, dst_base):
    """
    Preserve class structure while preprocessing jpegs
    """
    print(f"\n🔄 Preprocessing jpegs in: {src_base}")
    os.makedirs(dst_base, exist_ok=True)

    total = 0

    for class_name in os.listdir(src_base):
        class_src = os.path.join(src_base, class_name)
        if not os.path.isdir(class_src):
            continue

        class_dst = os.path.join(dst_base, class_name)
        os.makedirs(class_dst, exist_ok=True)

        files = glob.glob(f"{class_src}/*.jpeg") + glob.glob(f"{class_src}/*.JPEG")
        print(f"  {class_name}: {len(files)} jpeg files")

        for f in tqdm(files, leave=False):
            img = preprocess_image(f)
            if img is not None:
                Image.fromarray(img).save(
                    os.path.join(class_dst, os.path.basename(f))
                )
                total += 1

    print(f"✓ Total processed jpegs: {total}")
    return total


In [13]:
datasets = [
    ("MRI/tumor_binary", "MRI/tumor_binary_norm"),
    ("MRI/tumor_multiclass", "MRI/tumor_multiclass_norm"),
    ("MRI/MS", "MRI/ms_norm"),
    ("CT/stroke_binary", "CT/stroke_binary_norm"),
]

for src, dst in datasets:
    batch_preprocess_dataset_jpegs(
        f"{PROC_DIR}/{src}",
        f"{PROC_DIR}/{dst}"
    )



🔄 Preprocessing jpegs in: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/processed/MRI/tumor_binary
  tumor: 0 jpeg files


  normal: 0 jpeg files


✓ Total processed jpegs: 0

🔄 Preprocessing jpegs in: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/processed/MRI/tumor_multiclass
  Carcinoma: 225 jpeg files


  Schwannoma: 735 jpeg files


  Germinoma: 57 jpeg files


...

  Control: 0 jpeg files


✓ Total processed jpegs: 0

🔄 Preprocessing jpegs in: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/processed/CT/stroke_binary


  stroke: 0 jpeg files


  normal: 0 jpeg files


✓ Total processed jpegs: 0


## TRAIN/VAL/TEST SPLITS (70/15/15 Stratified)

In [15]:
import os
import glob
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [16]:
def collect_images(class_path):
    files = []
    for ext in ("*.png", "*.jpg", "*.jpeg"):
        files.extend(glob.glob(os.path.join(class_path, ext)))
    return files

In [17]:
def create_stratified_splits(norm_dir, split_dir, dataset_name):
    """
    Create balanced train/val/test splits (70/15/15) per class
    """
    print(f"\n🔄 Creating splits for {dataset_name}")

    base_out = os.path.join(split_dir, dataset_name)
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(base_out, split), exist_ok=True)

    class_files = {}
    for class_name in sorted(os.listdir(norm_dir)):
        class_path = os.path.join(norm_dir, class_name)
        if os.path.isdir(class_path):
            files = collect_images(class_path)
            if len(files) > 0:
                class_files[class_name] = files

    split_counts = []

    for class_name, files in tqdm(class_files.items(), desc="Splitting classes"):
        if len(files) < 5:
            print(f"⚠️ Skipping {class_name} (too few samples)")
            continue

        train_files, temp_files = train_test_split(
            files, train_size=0.7, random_state=42
        )
        val_files, test_files = train_test_split(
            temp_files, test_size=0.5, random_state=42
        )

        splits = {
            "train": train_files,
            "val": val_files,
            "test": test_files
        }

        for split_name, split_files in splits.items():
            out_dir = os.path.join(base_out, split_name, class_name)
            os.makedirs(out_dir, exist_ok=True)

            for f in split_files:
                shutil.copy(f, os.path.join(out_dir, os.path.basename(f)))

        split_counts.append({
            "dataset": dataset_name,
            "class": class_name,
            "total": len(files),
            "train": len(train_files),
            "val": len(val_files),
            "test": len(test_files)
        })

    df = pd.DataFrame(split_counts)
    df.to_csv(os.path.join(base_out, "split_stats.csv"), index=False)

    print(df.groupby("dataset")[["train", "val", "test"]].sum())
    return df

## Run splitting for ALL datasets

In [18]:
SPLIT_DIR = f"{BASE_DIR}/data/split"
PROC_DIR = f"{BASE_DIR}/data/processed"

datasets = [
    "MRI/tumor_binary_norm",
    "MRI/tumor_multiclass_norm",
    "MRI/ms_norm",
    "CT/stroke_binary_norm"
]

all_stats = []

for d in datasets:
    stats = create_stratified_splits(
        norm_dir=f"{PROC_DIR}/{d}",
        split_dir=SPLIT_DIR,
        dataset_name=d.replace("/", "_")
    )
    all_stats.append(stats)

final_stats = pd.concat(all_stats)
final_stats.to_csv(f"{BASE_DIR}/final_split_stats.csv", index=False)


🔄 Creating splits for MRI_tumor_binary_norm


Splitting classes: 100%|██████████| 2/2 [01:13<00:00, 36.52s/it]


                       train  val  test
dataset                                
MRI_tumor_binary_norm   2100  450   450

🔄 Creating splits for MRI_tumor_multiclass_norm


Splitting classes: 100%|██████████| 12/12 [02:37<00:00, 13.12s/it]


                           train   val  test
dataset                                     
MRI_tumor_multiclass_norm   5556  1194  1197

🔄 Creating splits for MRI_ms_norm


Splitting classes: 100%|██████████| 2/2 [01:26<00:00, 43.27s/it]


             train  val  test
dataset                      
MRI_ms_norm   2731  586   586

🔄 Creating splits for CT_stroke_binary_norm


Splitting classes: 100%|██████████| 2/2 [03:12<00:00, 96.24s/it]

                       train  val  test
dataset                                
CT_stroke_binary_norm   4654  997   999
